#  Project PrismCX – Emphasizing customer experience and segmentation across many dimensions.

### 📌 Objective
To develop a data-driven customer segmentation model for an IoT-based home automation company by leveraging RFM (Recency, Frequency, Monetary) analysis and the CORE (Convert, Optimize, Retain, Exit) strategy to improve customer engagement, retention, and marketing personalization.
- **Convert** – New or low-engagement customers
- **Optimize** – Recently active but moderate value
- **Retain** – Loyal and high-value customers
- **Exit** – Lapsed or at-risk customers

---

### 🧾 Dataset Description
The dataset used (`SmartHome_PrismCX_Segmentation.csv`) contains 50,000 transaction records with:
- Customer ID, Transaction Date, Transaction Value
- Product Category, Payment Method, Gender, Age Group, Region

---

### 🛠 Tools Used
- Python (Pandas, Datetime)
- Exported CSV for Tableau Visualizations

## Step 1: Load Transaction Data
We load the UrbanMart dataset and convert `TransactionDate` to datetime format so we can later compute recency. We also define a snapshot date as one day after the last transaction.


In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [2]:
df = pd.read_csv("SmartHome_PrismCX_Segmentation.csv")
print(df.shape)
print(df.info())
print(df.describe())

FileNotFoundError: [Errno 2] No such file or directory: 'SmartHome_PrismCX_Segmentation.csv'

In [1588]:
df = df.dropna(thresh=df.shape[1] - 2)

# Fill numerical columns with mean
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Fill categorical columns with mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [1590]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
df = df.dropna(subset=['TransactionDate'])
snapshot_date = df['TransactionDate'].max() + timedelta(days=1)

## Step 2: Calculate RFM Metrics
We calculate:
- **Recency**: Days since last purchase
- **Frequency**: Number of purchases
- **Monetary**: Total spending

This is done by grouping data by `CustomerID`.

In [1593]:
rfm = df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'TransactionID': 'count',                                     # Frequency
    'TransactionValue': ['sum', 'mean']                           # Monetary + Avg Value
}).reset_index()


In [1595]:
# Flatten MultiIndex columns
rfm.columns = ['CustomerID', 'Recency', 'TransactionCount', 'Monetary', 'AvgTransactionValue']


## Step 3: Analyze RFM Distribution
We print descriptive statistics and percentiles for Recency, Frequency, and Monetary. These values help us determine thresholds for churn risk, loyalty, and top spenders.

In [1598]:
# Add emojis, lines, and spacing for readability
print("🔁 ===== Recency (days since last purchase) =====")
print(rfm['Recency'].describe())

print("\n🔂 ===== Frequency (number of purchases) =====")
print(rfm['TransactionCount'].describe())

print("\n💸 ===== Monetary (total spending) =====")
print(rfm['Monetary'].describe())

# Print percentiles
print("\n🎯 ===== Key Percentiles =====")
for metric in ['Recency', 'TransactionCount', 'Monetary']:
    print(f"\n📊 {metric} percentiles:")
    print(rfm[metric].quantile([0.25, 0.5, 0.75, 0.9, 0.95]))


🔁 ===== Recency (days since last purchase) =====
count    2000.000000
mean       29.610000
std        28.727975
min         1.000000
25%         9.000000
50%        20.000000
75%        42.000000
max       215.000000
Name: Recency, dtype: float64

🔂 ===== Frequency (number of purchases) =====
count    2000.000000
mean       25.000000
std         5.057152
min        11.000000
25%        21.000000
50%        25.000000
75%        28.000000
max        45.000000
Name: TransactionCount, dtype: float64

💸 ===== Monetary (total spending) =====
count    2.000000e+03
mean     6.335203e+07
std      1.468154e+07
min      1.909434e+07
25%      5.328737e+07
50%      6.268898e+07
75%      7.276311e+07
max      1.147111e+08
Name: Monetary, dtype: float64

🎯 ===== Key Percentiles =====

📊 Recency percentiles:
0.25     9.0
0.50    20.0
0.75    42.0
0.90    68.0
0.95    85.0
Name: Recency, dtype: float64

📊 TransactionCount percentiles:
0.25    21.0
0.50    25.0
0.75    28.0
0.90    32.0
0.95    34.0
Nam

## Step X: Automated RFM Scoring Based on Data Quantiles

Instead of hardcoding or manually assigning RFM scores using `qcut`, we automate scoring using the dataset's actual distribution.  
We calculate the 25th, 50th, and 75th percentiles for Recency, Frequency, and Monetary, then assign scores:

- **Score 4** = Best performers
- **Score 1** = Least engaged

For **Recency**, lower is better (recent shoppers), so scoring is reversed:
- Recency ≤ 25th percentile → Score 4
- Recency > 75th percentile → Score 1

For **Frequency** and **Monetary**, higher is better:
- Frequency ≥ 75th percentile → Score 4
- Frequency ≤ 25th percentile → Score 1

This makes the scoring system **adaptive to any dataset** and removes reliance on `qcut`, which can break with outliers or skewed data.


In [1601]:
def score_rfm(value, quantiles, reverse=False):
   
    if value <= quantiles[0.25]:
        return 4 if reverse else 1
    elif value <= quantiles[0.50]:
        return 3 if reverse else 2
    elif value <= quantiles[0.75]:
        return 2 if reverse else 3
    else:
        return 1 if reverse else 4


r_quartiles = rfm['Recency'].quantile([0.25, 0.50, 0.75])
f_quartiles = rfm['TransactionCount'].quantile([0.25, 0.50, 0.75])
m_quartiles = rfm['Monetary'].quantile([0.25, 0.50, 0.75])


rfm['R_Score'] = rfm['Recency'].apply(score_rfm, args=(r_quartiles, True))
rfm['F_Score'] = rfm['TransactionCount'].apply(score_rfm, args=(f_quartiles, False))
rfm['M_Score'] = rfm['Monetary'].apply(score_rfm, args=(m_quartiles, False))


## Step 5: Map RFM Scores to CORE Segments
We define a custom segmentation framework called **CORE**:
- **Retain**: High R, F, and M scores (3+)
- **Optimize**: Recent but not top-tier customers (R ≥ 3 with F or M = 2)
- **Convert**: Mid Recency and low Frequency & Monetary
- **Exit**: Everyone else, especially low Recency

This logic helps businesses target marketing efforts strategically.


In [1604]:
def classify_core(r, f, m):
    if r >= 3 and f >= 3 and m >= 3:
        return 'Retain'
    elif r >= 3 and (f == 2 or m == 2):
        return 'Optimize'
    elif r == 2 and f <= 2 and m <= 2:
        return 'Convert'
    else:
        return 'Exit'

rfm['CORE_Segment'] = rfm.apply(lambda row: classify_core(row['R_Score'], row['F_Score'], row['M_Score']), axis=1)


## Step 6: Add Customer Flags
We add important behavioral tags:
- **Churn_Risk**: Recency > 90 days
- **Top_Spender**: Top 10% in Monetary value
- **IsLoyalCustomer**: Customers with 5 or more transactions

These help further classify customers beyond RFM alone.


In [1607]:
rfm['Churn_Risk'] = rfm['Recency'].apply(lambda x: 'Yes' if x > 90 else 'No')

top_spend_threshold = rfm['Monetary'].quantile(0.90)
rfm['Top_Spender'] = rfm['Monetary'].apply(lambda x: 'Yes' if x >= top_spend_threshold else 'No')


## Step 7: Average Days Between Purchases
This metric helps us understand how frequently a customer shops on average.

In [1610]:
purchase_intervals = df.sort_values(by=['CustomerID', 'TransactionDate']).groupby('CustomerID')['TransactionDate'].diff().dt.days
avg_days_between = purchase_intervals.groupby(df['CustomerID']).mean().reset_index()
avg_days_between.columns = ['CustomerID', 'AvgDaysBetweenPurchases']

## Step 8: To determine if a custmer is a loyal customer or not
This metric helps us understand if the customer is a regular or not by define the number of purchases to exceed

In [1613]:
rfm['IsLoyalCustomer'] = rfm['TransactionCount'].apply(lambda x: 'Yes' if x >= 5 else 'No')


## Step 9: Merge All Features and Export Final Dataset
We merge the RFM + CORE + behavior flags back into the full transaction-level data and export it as a CSV to be used in Tableau.

In [1616]:
df_final = pd.merge(df, rfm, on='CustomerID', how='left')
df_final.to_csv("UrbanMart_CORE_Segmentation_Enhanced.csv", index=False)

In [1618]:
print(df_final.head(10))


  TransactionID CustomerID TransactionDate  TransactionValue ProductCategory  \
0   TRANS000000  CUST01574      2023-04-09            467907           Sport   
1   TRANS000001  CUST01836      2024-08-05           2713789          Beauty   
2   TRANS000002  CUST01291      2024-04-07           4539248         Fashion   
3   TRANS000003  CUST00245      2024-06-12           3058426            Home   
4   TRANS000004  CUST00163      2023-12-18           3304874         Fashion   
5   TRANS000005  CUST00331      2024-11-24           1556151     Electronics   
6   TRANS000006  CUST01956      2023-03-10           1664460            Home   
7   TRANS000007  CUST01713      2023-04-10           4194384         Fashion   
8   TRANS000008  CUST00112      2024-09-03           1184496           Sport   
9   TRANS000009  CUST01726      2024-06-09           3314052            Home   

   PaymentMethod CustomerGender CustomerAgeGroup    Region  Recency  \
0  Bank Transfer         Female              >50